In [1]:
import android_manager
import poke_manager
import win32gui
import win32con
import pyautogui
import time
from ppadb.client import Client as AdbClient
from ppadb.device import Device as AdbDevice
import importlib
import os
from pathlib import Path

In [3]:
importlib.reload(android_manager)
importlib.reload(poke_manager)

#scrcpy로 Device가 실행된 상태
cur_client = AdbClient(host='localhost', port=5037)
#serial number:
#ce10171ab2312a0d04 for s8
#ce071717d4035622047e for note8
pm_s8 = poke_manager.Poke_device(cur_client.device('ce10171ab2312a0d04'), 2)
pm_note8 = poke_manager.Poke_device(cur_client.device('ce071717d4035622047e'), 1)
#my-pm은 둘 중 하나를 복사
#my_pm = pm_s8
my_pm = pm_note8
#always-on-top 옵션 주기 > 나중에는 클래스 안에서 구현. 
pm_devices = [d for d in [pm_s8, pm_note8] if d is not None]

(3110, 1335)
Deviced SM-G950N is connedted
Hwnd: 264490
(3470, 1335)
Deviced SM-N950N is connedted
Hwnd: 3803324


In [96]:
win32api.GetSystemMetrics(win32con.SM_CXSIZEFRAME)

4

In [54]:
import win32api

title_bar_height = win32api.GetSystemMetrics(win32con.SM_CYCAPTION)
border_width = win32api.GetSystemMetrics(win32con.SM_CXSIZEFRAME)
border_height = win32api.GetSystemMetrics(win32con.SM_CYSIZEFRAME)
title_bar_size = (border_width * 2, border_height + title_bar_height)

(8, 27)

In [60]:
output = my_pm.shell('wm size')
print(output)
print("Screen size: " + output.split()[-1])

Physical size: 1440x2960
Override size: 1080x2220

Screen size: 1080x2220


In [61]:
a = '1080x2220'
print(a.split('x'))

['1080', '2220']


In [106]:
#선물열기 자동화
profile_xy = (137, 1892)
profile_friend_xy = (755, 163)
profile_sort_xy = (934, 1921)
profile_sort_present_xy = (927, 1534)

my_pm.confidence = 0.90

while True:
    my_pm.automation_by_image('bcb', profile_xy, new_status='profile')
    my_pm.automation_by_image('profile_me', profile_friend_xy, cur_status='profile', new_status='profile_friend')
    my_pm.automation_by_image('profile_friend', no_click=True, cur_status='profile', new_status='profile_friend')
    if my_pm.status == 'profile_friend' and not my_pm.automation_by_image('present', cur_status='profile_friend', new_status='open_present'):
        print("오픈할 선물이 없음")
        #선물로 소팅되어 있는 경우 나갔다가 들어옴
        if my_pm.automation_by_image('profile_friend_sort_present_desc', target_img='profile_friend_x_green', new_status = 'initial'):
            pass
        else:
            #선물로 소팅하기 
            my_pm.click_xy(profile_sort_xy, is_device_xy=True)
            my_pm.click_xy(profile_sort_present_xy, is_device_xy=True)
            time.sleep(2)
            my_pm.automation_by_image('sort_present_asc')
    if my_pm.automation_by_image('open_present_complete', target_img='x_white', cur_status='after_open_present'):
        my_pm.status = 'finished_open_present'
        break
    my_pm.automation_by_image('present_to_open', cur_status='open_present')
    
    my_pm.automation_by_image('open_cmd', new_status='after_open_present')
    my_pm.automation_by_image('exchange_new_record', target_img='x_white', cur_status='after_open_present')
    my_pm.automation_by_image('profile_friend_interact_x', cur_status='after_open_present', new_status='profile_friend')
    my_pm.automation_by_image('initial_seemore', target_img='x_green', new_status='initial')
    time.sleep(1)

KeyboardInterrupt: 

In [39]:
my_pm.automation_by_image('open_present_complete', target_img='x_white')

Found open_present_complete at Point(x=1136, y=490)
Img: open_present_complete, T_img: x_white
Click: Point(x=1206, y=880)(is_tap_xy: False)


True

In [32]:
# initial_seemore
my_pm.automation_by_image('x_green')

False

In [19]:
my_pm.automation_by_image('profile_friend_interact_x')

Found profile_friend_interact_x at Point(x=1207, y=865)
Img: profile_friend_interact_x, T_img: None
Click: Point(x=1207, y=865)(is_tap_xy: False)


True

In [11]:
my_pm.automation_by_image('profile_me_adv_note')

Found profile_me_adv_note at Point(x=1263, y=672)
Img: profile_me_adv_note, T_img: None
Click: Point(x=1263, y=672)(is_tap_xy: False)


True

In [8]:
#한영 자동전환
keyboard_language_change_xy = (329,1966)
#수원
text = 'tndnjs&vkxmsj vhzptahs0'
#한글로 전환
my_pm.automation_by_image('keyboard_qwerty', device_xy=keyboard_language_change_xy)
my_pm.device.input_text(text)
my_pm.automation_by_image('keyboard_confirm')

False

In [97]:
my_pm.status = 'after_exchange_pokemon'

In [13]:
#교환 자동화
first_pokemon_in_list = (191, 734)
exchange_special_change_no = (523, 1302)
my_pm.confidence = 0.8

while True:
    my_pm.automation_by_image('profile_friend_detail_exchange', new_status='before_exchange')
    my_pm.automation_by_image('exchange_pokemon_list', device_xy=first_pokemon_in_list)
    my_pm.automation_by_image('exchange_next_cmd')
    my_pm.automation_by_image('exchange_100_cost', target_img='exchange_confirm_cmd', new_status='after_exchange')
    my_pm.automation_by_image('exchange_special_change', exchange_special_change_no)
    my_pm.automation_by_image('x_green', cur_status='after_exchange', new_status='before_exchange')

Found exchange_pokemon_list at Point(x=3623, y=1473)
Img: exchange_pokemon_list, T_img: None
Click: (191, 734)(is_tap_xy: True)
Found exchange_next_cmd at Point(x=3651, y=1939)
Img: exchange_next_cmd, T_img: None
Click: Point(x=3651, y=1939)(is_tap_xy: False)
Found exchange_100_cost at Point(x=3585, y=1737)
Img: exchange_100_cost, T_img: exchange_confirm_cmd
Click: Point(x=3508, y=1736)(is_tap_xy: False)
prev status: before_exchange, new status:after_exchange
Found exchange_100_cost at Point(x=3585, y=1737)
Found exchange_100_cost at Point(x=3585, y=1737)
Found exchange_100_cost at Point(x=3585, y=1738)
Found x_green at Point(x=3656, y=2013)
Img: x_green, T_img: None
Click: Point(x=3656, y=2013)(is_tap_xy: False)
prev status: after_exchange, new status:before_exchange
Found profile_friend_detail_exchange at Point(x=3764, y=1916)
Img: profile_friend_detail_exchange, T_img: None
Click: Point(x=3764, y=1916)(is_tap_xy: False)
prev status: before_exchange, new status:before_exchange
Found 

error: (1400, 'GetWindowRect', '잘못된 창 핸들입니다.')

In [12]:
my_pm.status

'before_exchange'

In [11]:
# my_pm.click_xy((3132, 1887))
my_pm.automation_by_image('x_green')

Found x_green at Point(x=3656, y=2013)
Img: x_green, T_img: None
Click: Point(x=3656, y=2013)(is_tap_xy: False)


True

In [ ]:
import subprocess
subprocess.run()
pyautogui.displayMousePosition()

In [102]:
#언어 입력 관련
text = 'tjdnf&!vkxmsjvhzptahs0'
pm_note8.device.input_text(text)

''